In [ ]:
import app
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score




# Do extensions code below
dataset = pd.read_csv('admissions_data.csv')

dataset = dataset.drop(['Serial No.'], axis = 1)

labels = dataset.iloc[:,-1]
features = dataset.iloc[:,0:-1]

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)
  
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns

ct = ColumnTransformer([("only numeric", StandardScaler(),numerical_columns)], remainder='passthrough')

features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

features_train_scaled = pd.DataFrame(features_train_scaled, columns=features_train.columns)
features_test_scaled = pd.DataFrame(features_test_scaled, columns=features_test.columns)

def design_model(features, learning_rate):
  my_model = Sequential()
  input = InputLayer(input_shape = (features.shape[1], ))
  my_model.add(input)
  my_model.add(Dense(64, activation='relu'))
  my_model.add(layers.Dropout(0.2))
  my_model.add(layers.Dense(128, activation='relu'))
  my_model.add(layers.Dropout(0.1))
  my_model.add(layers.Dense(24, activation='relu'))
  my_model.add(layers.Dropout(0.3))
  my_model.add(Dense(1))
  opt = Adam(learning_rate = learning_rate)
  my_model.compile(loss='mse',  metrics=['mae'], optimizer=opt)
  return my_model

def fit_model(my_model, features_train, labels_tain, learning_rate, num_epochs):
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 20)
  
  history = my_model.fit(features_train, labels_train, epochs=num_epochs, batch_size= 2, verbose=0, validation_split = 0.2, callbacks = [es])
  return history

def fig(history):
    # plot learning curves
  fig = plt.figure()
  ax1 = fig.add_subplot(2, 1, 1)
  ax1.plot(history.history['mae'])
  ax1.plot(history.history['val_mae'])
  ax1.set_title('model mae')
  ax1.set_ylabel('MAE')
  ax1.set_xlabel('epoch')
  ax1.legend(['train', 'validation'], loc='upper left')
  
    # Plot loss and val_loss over each epoch
  ax2 = fig.add_subplot(2, 1, 2)
  ax2.plot(history.history['loss'])
  ax2.plot(history.history['val_loss'])
  ax2.set_title('model loss')
  ax2.set_ylabel('loss')
  ax2.set_xlabel('epoch')
  ax2.legend(['train', 'validation'], loc='upper left')



learning_rate = 0.1
num_epochs = 40

print("Results of model:")
history = fit_model(design_model(features_train, learning_rate), features_train, labels_train, learning_rate, num_epochs)

# print(my_model.summary())
# val_mse, val_mae = Sequential.evaluate(my_model)
# val_mse, val_mae = my_model.evaluate(features_train, labels_train, verbose=0)
my_model = design_model(features_train, learning_rate)
mse, mae = my_model.evaluate(features_train, labels_train, verbose=1)
val_mse, val_mae = my_model.evaluate(features_test, labels_test, verbose=0)
predicted_values = my_model.predict(features_test) 

dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(features_train, labels_train)
y_pred = dummy_regr.predict(features_test)
MAE_baseline = mean_absolute_error(labels_test, y_pred)

print("MSE: ", val_mse)
print("MAE: ", val_mae)
print("R SQUARED: ", r2_score(labels_test, predicted_values))
print("MEAN: ", MAE_baseline)
# print(val_mse, val_mae)
# my_model = design_model(features_train_scaled)
# print(my_model.summary())
# print(val_mse, val_mae)

# print(features_train_scaled.describe())
# print(features_test_scaled.describe())
# print(dataset.head())
# if you decide to do the Matplotlib extension, you must save your plot in the directory by uncommenting the line of code below

plt.tight_layout()
plt.savefig('static/images/my_plots.png')
plt.show()